In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

In [0]:
SS = SparkSession.builder.appName('Logit').getOrCreate()

In [0]:
df_t = SS.read.format('libsvm').load('/FileStore/tables/sample_libsvm_data.txt')

In [0]:
train,test = df_t.randomSplit([0.7,0.3])

In [0]:
logit = LogisticRegression()

In [0]:
logit_trained = logit.fit(train)

In [0]:
logit_eval = logit_trained.evaluate(test)

In [0]:
logit_eval.areaUnderROC

Out[9]: 1.0

In [0]:
logit_eval.accuracy

Out[10]: 1.0

In [0]:
pred = logit_trained.transform(test).select(['label','features','prediction'])

In [0]:
pred.show()

+-----+--------------------+----------+
label| features|prediction|
+-----+--------------------+----------+
 0.0|(692,[100,101,102...| 0.0|
 0.0|(692,[123,124,125...| 0.0|
 0.0|(692,[123,124,125...| 0.0|
 0.0|(692,[124,125,126...| 0.0|
 0.0|(692,[124,125,126...| 0.0|
 0.0|(692,[124,125,126...| 0.0|
 0.0|(692,[126,127,128...| 0.0|
 0.0|(692,[126,127,128...| 0.0|
 0.0|(692,[126,127,128...| 0.0|
 0.0|(692,[127,128,129...| 0.0|
 0.0|(692,[150,151,152...| 0.0|
 0.0|(692,[153,154,155...| 0.0|
 1.0|(692,[97,98,99,12...| 1.0|
 1.0|(692,[123,124,125...| 1.0|
 1.0|(692,[124,125,126...| 1.0|
 1.0|(692,[124,125,126...| 1.0|
 1.0|(692,[125,126,153...| 1.0|
 1.0|(692,[127,128,129...| 1.0|
 1.0|(692,[128,129,130...| 1.0|
 1.0|(692,[128,129,130...| 1.0|
+-----+--------------------+----------+
only showing top 20 rows

In [0]:
SS1 = SparkSession.builder.appName('Titanic').getOrCreate()

In [0]:
df = SS1.read.csv('/FileStore/tables/titanic.csv',inferSchema=True,header=True)

In [0]:
df.columns

Out[15]: ['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [0]:
df1 = df.na.drop()

In [0]:
from pyspark.ml.feature import VectorAssembler,OneHotEncoder,StringIndexer
from pyspark.ml import Pipeline

In [0]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='Sex_index')
gender_onehot = OneHotEncoder(inputCol='Sex_index',outputCol='Gender_one')

Embark_indexed = StringIndexer(inputCol='Embarked',outputCol='Embarked_index')
Embark_hot = OneHotEncoder(inputCol='Embarked_index',outputCol='Embarked_one')

In [0]:
Logit = LogisticRegression(labelCol='Survived')

In [0]:
assembler = VectorAssembler(inputCols=['Pclass', 'Gender_one', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_one'],outputCol='features')

In [0]:
pipe = Pipeline(stages=[gender_indexer,gender_onehot,Embark_indexed,Embark_hot,assembler,Logit])

In [0]:
train,test = df1.randomSplit([0.7,0.3])

In [0]:
LG = pipe.fit(train)

In [0]:
pred = LG.transform(test)

In [0]:
results = pred.select(['Survived','features','rawPrediction','probability','prediction']).show()

+--------+--------------------+--------------------+--------------------+----------+
Survived| features| rawPrediction| probability|prediction|
+--------+--------------------+--------------------+--------------------+----------+
 0|(8,[0,2,5,6],[1.0...|[1.27956986107158...|[0.78237654836564...| 0.0|
 1|[1.0,1.0,58.0,0.0...|[-1.3950306513881...|[0.19860585631731...| 1.0|
 0|[1.0,0.0,45.0,1.0...|[0.76828286421846...|[0.68314932578298...| 0.0|
 1|[2.0,1.0,29.0,0.0...|[-2.0336736537915...|[0.11571249138669...| 1.0|
 1|[1.0,1.0,23.0,3.0...|[-2.2341851794869...|[0.09672237610542...| 1.0|
 0|[1.0,0.0,46.0,1.0...|[0.79404023737319...|[0.68869818886878...| 0.0|
 0|[1.0,0.0,21.0,0.0...|[-0.0624512060747...|[0.48439227086274...| 1.0|
 0|[1.0,0.0,37.0,1.0...|[0.27793392413413...|[0.56903962530310...| 0.0|
 0|(8,[0,2,5,7],[1.0...|[-0.9636613647012...|[0.27614572785657...| 1.0|
 0|[2.0,0.0,36.5,0.0...|[2.22384332840815...|[0.90237030976540...| 0.0|
 1|[1.0,1.0,22.0,1.0...|[-3.4101598994173...|[0.03197944715085...| 1.0|
 0|(8,[0,2,5,7],[1.0...|[0.76623883179124...|[0.68270671649419...| 0.0|
 1|[1.0,1.0,58.0,0.0...|[-1.8284208335592...|[0.13842650432156...| 1.0|
 1|(8,[0,2,5,7],[1.0...|[-0.1313212408756...|[0.46721678918426...| 1.0|
 1|[1.0,0.0,38.0,1.0...|[0.38430423143764...|[0.59491081033042...| 0.0|
 1|[1.0,1.0,35.0,1.0...|[-2.6575770066238...|[0.06552353771146...| 1.0|
 1|[1.0,0.0,37.0,1.0...|[0.75262029018000...|[0.67974937902700...| 0.0|
 1|[1.0,1.0,30.0,0.0...|[-2.8848514693199...|[0.05290750953149...| 1.0|
 1|[1.0,1.0,58.0,0.0...|[-0.7467836849264...|[0.32152252328409...| 1.0|
 0|[1.0,0.0,37.0,0.0...|[0.17396794541063...|[0.54338262745787...| 0.0|
+--------+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
evalu= BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [0]:
evalu.evaluate(pred)

Out[28]: 0.7766081871345029

In [0]:
SS2 = SparkSession.builder.appName('test').getOrCreate()

In [0]:
df = SS2.read.csv('/FileStore/tables/customer_churn.csv',inferSchema=True,header=True)

In [0]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|
 Jessica Williams|48.0| 10356.02| 0| 5.12| 8.0|2009-03-03 23:13:37|6187 Olson Mounta...| Kelly-Warren| 1|
 Eric Butler|44.0| 11331.58| 1| 5.23| 11.0|2016-12-05 03:35:43|4846 Savannah Roa...| Reynolds-Sheppard| 1|
 Zachary Walsh|32.0| 9885.12| 1| 6.92| 9.0|2006-03-09 14:50:20|25271 Roy Express...| Singh-Cole| 1|
 Ashlee Carr|43.0| 14062.6| 1| 5.46| 11.0|2011-09-29 05:47:23|3725 Caroline Str...| Lopez PLC| 1|
 Jennifer Lynch|40.0| 8066.94| 1| 7.11| 11.0|2006-03-28 15:42:45|363 Sandra Lodge ...| Reed-Martinez| 1|
 Paula Harris|30.0| 11575.37| 1| 5.22| 8.0|2016-11-13 13:13:01|Unit 8120 Box 916...|Briggs, Lamb and ...| 1|
 Bruce Phillips|45.0| 8771.02| 1| 6.64| 11.0|2015-05-28 12:14:03|Unit 1895 Box 094...| Figueroa-Maynard| 1|
 Craig Garner|45.0| 8988.67| 1| 4.84| 11.0|2011-02-16 08:10:47|897 Kelley Overpa...| Abbott-Thompson| 1|
 Nicole Olson|40.0| 8283.32| 1| 5.1| 13.0|2012-11-22 05:35:03|11488 Weaver Cape...|Smith, Kim and Ma...| 1|
 Harold Griffin|41.0| 6569.87| 1| 4.3| 11.0|2015-03-28 02:13:44|1774 Peter Row Ap...|Snyder, Lee and M...| 1|
 James Wright|38.0| 10494.82| 1| 6.81| 12.0|2015-07-22 08:38:40|45408 David Path ...| Sanders-Pierce| 1|
 Doris Wilkins|45.0| 8213.41| 1| 7.35| 11.0|2006-09-03 06:13:55|28216 Wright Moun...|Andrews, Adams an...| 1|
Katherine Carpenter|43.0| 11226.88| 0| 8.08| 12.0|2006-10-22 04:42:38|Unit 4948 Box 481...|Morgan, Phillips ...| 1|
 Lindsay Martin|53.0| 5515.09| 0| 6.85| 8.0|2015-10-07 00:27:10|69203 Crosby Divi...| Villanueva LLC| 1|
 Kathy Curry|46.0| 8046.4| 1| 5.69| 8.0|2014-11-06 23:47:14|9569 Caldwell Cre...|Berry, Orr and Ca...| 1|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [0]:
from pyspark.ml.feature import VectorAssembler,OneHotEncoder,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
df.columns

Out[32]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
VA = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'],outputCol='features')

In [0]:
df_Data = VA.transform(df).select(['features','Churn'])

In [0]:
df_Data.show()

+--------------------+-----+
 features|Churn|
+--------------------+-----+
[42.0,11066.8,0.0...| 1|
[41.0,11916.22,0....| 1|
[38.0,12884.75,0....| 1|
[42.0,8010.76,0.0...| 1|
[37.0,9191.58,0.0...| 1|
[48.0,10356.02,0....| 1|
[44.0,11331.58,1....| 1|
[32.0,9885.12,1.0...| 1|
[43.0,14062.6,1.0...| 1|
[40.0,8066.94,1.0...| 1|
[30.0,11575.37,1....| 1|
[45.0,8771.02,1.0...| 1|
[45.0,8988.67,1.0...| 1|
[40.0,8283.32,1.0...| 1|
[41.0,6569.87,1.0...| 1|
[38.0,10494.82,1....| 1|
[45.0,8213.41,1.0...| 1|
[43.0,11226.88,0....| 1|
[53.0,5515.09,0.0...| 1|
[46.0,8046.4,1.0,...| 1|
+--------------------+-----+
only showing top 20 rows

In [0]:
train,test = df_Data.randomSplit([0.7,0.3])

In [0]:
LR = LogisticRegression(labelCol='Churn')

In [0]:
LR_data = LR.fit(train)

In [0]:
pred  = LR_data.transform(test)

In [0]:
ev = LR_data.evaluate(test)

In [0]:
ev.precisionByLabel

Out[40]: [0.91701244813278, 0.717948717948718]

In [0]:
ev.recallByLabel

Out[41]: [0.9525862068965517, 0.5833333333333334]

In [0]:
ev.recallByThreshold.show()

+------------------+--------------------+
 threshold| recall|
+------------------+--------------------+
0.9851880450999456|0.020833333333333332|
0.9647194288202463|0.041666666666666664|
0.9514284508495165| 0.0625|
0.9452143644263299| 0.08333333333333333|
0.9334147803141272| 0.10416666666666667|
0.9236876491667142| 0.10416666666666667|
0.8713286969183844| 0.125|
0.8663497639238763| 0.14583333333333334|
0.8587285205405021| 0.16666666666666666|
0.8504509563311148| 0.1875|
0.8171470508473742| 0.20833333333333334|
0.8108679619605512| 0.22916666666666666|
 0.806022670250349| 0.25|
0.7928016052953909| 0.2708333333333333|
0.7919048365608121| 0.2708333333333333|
0.7851176452184312| 0.2916666666666667|
0.7558117606419541| 0.3125|
 0.742929240647241| 0.3333333333333333|
0.7427889043365714| 0.3333333333333333|
0.7427385865982026| 0.3541666666666667|
+------------------+--------------------+
only showing top 20 rows

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
pred.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|Churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[22.0,11254.38,1....| 0|[4.74588375941614...|[0.99138743953719...| 0.0|
[26.0,8939.61,0.0...| 0|[6.47027466919376...|[0.99845359480599...| 0.0|
[27.0,8628.8,1.0,...| 0|[5.55752830531213...|[0.99615652810797...| 0.0|
[28.0,11245.38,0....| 0|[3.73776707131166...|[0.97674639958736...| 0.0|
[29.0,12711.15,0....| 0|[5.48824045317875...|[0.99588191504650...| 0.0|
[29.0,13255.05,1....| 0|[4.3308238884942,...|[0.98701414764046...| 0.0|
[30.0,6744.87,0.0...| 0|[3.41687787642081...|[0.96822786705817...| 0.0|
[30.0,7960.64,1.0...| 1|[3.18853550312543...|[0.96040056123352...| 0.0|
[30.0,10183.98,1....| 0|[2.93691683149155...|[0.94964148611845...| 0.0|
[31.0,5304.6,0.0,...| 0|[3.29965132391194...|[0.96441684715020...| 0.0|
[31.0,7073.61,0.0...| 0|[2.99587893293720...|[0.95238760315888...| 0.0|
[31.0,10058.87,1....| 0|[4.54438584380911...|[0.98948504210882...| 0.0|
[31.0,10182.6,1.0...| 0|[4.95440740190116...|[0.99299712786600...| 0.0|
[32.0,6367.22,1.0...| 0|[3.03377185099766...|[0.95407671743806...| 0.0|
[32.0,7896.65,0.0...| 0|[3.33562821981633...|[0.96563104783863...| 0.0|
[32.0,8011.38,0.0...| 0|[1.89188342561941...|[0.86897012843599...| 0.0|
[32.0,8575.71,0.0...| 0|[3.78745062943200...|[0.97784852368956...| 0.0|
[32.0,9885.12,1.0...| 1|[1.75536883147209...|[0.85262868975768...| 0.0|
[32.0,11540.86,0....| 0|[6.98471425521604...|[0.99907492871138...| 0.0|
[32.0,12547.91,0....| 0|[0.33025597476631...|[0.58182165814180...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
BCE = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

In [0]:
BCE.evaluate(pred)

Out[46]: 0.7679597701149425

In [0]:
LR1 = LogisticRegression(labelCol='Churn')

In [0]:
LR_data = LR1.fit(train)

In [0]:
ss = SparkSession.builder.appName('test').getOrCreate()

In [0]:
test = ss.read.csv('/FileStore/tables/new_customers.csv',inferSchema=True,header=True)

In [0]:
test.show()

VA1= VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'],outputCol='features')

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+

In [0]:
test_data = VA1.transform(test)

In [0]:
pred  = LR_data.transform(test_data)

In [0]:
pred.select(['Company','prediction']).show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+